In [5]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from urllib.request import urlopen
from PIL import Image
import timm
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from sklearn.model_selection import train_test_split
import torch.nn as nn
import random
import torchmetrics
from torch.nn.modules.batchnorm import _BatchNorm
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [7]:
class_list={
    'background':0,
    'Tumor':1,
    'Stroma':2,
    'Immune cells':3,
    'Necrosis':4,
    'alveoli':5,
    'Other':6
}
params={'image_size':128,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':32,
        'epochs':200,
        'n_classes':len(class_list),
        'inch':3,
        }

In [20]:
trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

def transback(data:Tensor) -> Tensor:
    return data / 2 + 0.5

class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,parmas, images,label):
        
        self.images = images
        self.args=parmas
        self.label=label
        
    def trans(self,image):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            image = transform(image)
            
        return image
    
    def __getitem__(self, index):
        image=self.images[index]
        label=self.label[index]
        image = self.trans(image)
        return image,label
    
    def __len__(self):
        return len(self.images)


train_image_label=[]
train_image_path=[]
val_image_label=[]
val_image_path=[]
for dataset_name in range(2):
    for i in tqdm(range(len(class_list))):
        if dataset_name==0:
            image_list=glob(f'../../data/MIHIC/train/{list(class_list.keys())[i]}/*.png')
            train_image_path.extend(image_list)
            for k in range(len(image_list)):
                train_image_label.append(i)
        else:
            image_list=glob(f'../../data/MIHIC/val/{list(class_list.keys())[i]}/*.png')
            val_image_path.extend(image_list)
            for k in range(len(image_list)):
                val_image_label.append(i)

train_images=torch.zeros((len(train_image_path),params['inch'],params['image_size'],params['image_size']))
val_images=torch.zeros((len(val_image_path),params['inch'],params['image_size'],params['image_size']))
for i in tqdm(range(len(train_image_path))):
    train_images[i]=trans(Image.open(train_image_path[i]).convert('RGB').resize((params['image_size'],params['image_size'])))
for i in tqdm(range(len(val_image_path))):
    val_images[i]=trans(Image.open(val_image_path[i]).convert('RGB').resize((params['image_size'],params['image_size'])))
    
# X_train, X_test, y_train, y_test = train_test_split(train_images, image_label, test_size=0.2, random_state=42)
# train_dataset=CustomDataset(params,X_train,F.one_hot(torch.tensor(y_train)).to(torch.int64))
# val_dataset=CustomDataset(params,X_test,F.one_hot(torch.tensor(y_test)).to(torch.int64))
# dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True)
# val_dataloader=DataLoader(val_dataset,batch_size=params['batch_size'],shuffle=True)

 23%|██▎       | 45444/195001 [01:43<05:42, 437.02it/s]


KeyboardInterrupt: 

In [19]:
len(train_image_label)

195001

In [9]:
class_list.keys()

dict_keys(['background', 'Tumor', 'Stroma', 'Immune cells', 'Necrosis', 'alveoli', 'Other'])